In [ ]:
import pandas as pd
import numpy as np

### The First part of this script is about the setup of the data collected

In [ ]:
# Read data from the file
data = pd.read_csv('D:\OneDrive - Texas Tech University\PUBLICATIONS\In Progress\complex_supply_chain\data_output\data.csv')
#data = pd.read_csv(r"C:\Users\gar14685\OneDrive - Texas Tech University\PUBLICATIONS\In Progress\complex_supply_chain\data_output\data.csv")
#data = pd.read_csv('/Users/memesmacbookair/Library/CloudStorage/OneDrive-TexasTechUniversity/PUBLICATIONS/In Progress/complex_supply_chain/data_output/data.csv')

# Deleting observations without answers to the Choice Experiments
data[['Q30','Q32','Q34','Q36','Q38','Q40','Q42','Q44','Q46','Q48','Q50','Q52']] = data[['Q30','Q32','Q34','Q36','Q38','Q40','Q42','Q44','Q46','Q48','Q50','Q52']].fillna(value=0)

data = data.drop(data[(data['Q30'] == 0) | (data['Q32'] == 0) | (data['Q34'] == 0) | (data['Q36'] == 0) | (data['Q38'] == 0) | (data['Q40'] == 0) | (data['Q42'] == 0) | (data['Q44'] == 0) | (data['Q46'] == 0) | (data['Q48'] == 0) | (data['Q50'] == 0) | (data['Q52'] == 0)].index)

data = data.reset_index()

In [ ]:
# Generate a range of integers for the ID column
data = data.assign(ID=range(1, len(data)+1))

#print(data['ID'].tail())
data.loc[:,['ID']]

In [ ]:
# Repeat each row 36 times
data = pd.concat([data]*36, ignore_index=True).sort_values(by='ID')

# Create a column with the values 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12 repeated three times
data["Set"] = [(i//3)%12 + 1 for i in range(len(data))]

#data.loc[:,['ID']]

In [ ]:
# sort data by Block and Alt
data2 = data.sort_values(by=['Block', 'ID', 'Set',])

#data2.iloc[:40,-8:]

In [ ]:
# Create a column "Alt" with values 1, 2, 3, repeating for each row
data2["Alt"] = [i%3 + 1 for i in range(len(data2))]

#data2.iloc[:40,-8:]

In [ ]:
# Create a column "Q" with values i1, i2, i3, for i = abcdefghijkl, repeating for each row
n_rows = data2.shape[0]

# Define the values for column "Q"
values = [f'{letter}{i}' for letter in 'abcdefghijkl' for i in range(1, 4)] * (n_rows // 12 + 1)

# Create the column "Q" with the values
data2['Q'] = values[:n_rows]

In [ ]:
# assigning values of 1 to the choice selected by respondents    
data2['Choice'] = 0

data2.loc[data2.query('(Q == "a1" & Q30 == "Opción 1") | (Q == "a2" & Q30 == "Opción 2") | (Q == "a3" & Q30 == "Ninguno") | \
    (Q == "b1" & Q32 == "Opción 1")| (Q == "b2" & Q32 == "Opción 2") | (Q == "b3" & Q32 == "Ninguno") | (Q == "c1" & Q34 == "Opción 1")| \
        (Q == "c2" & Q34 == "Opción 2") | (Q == "c3" & Q34 == "Ninguno") | (Q == "d1" & Q36 == "Opción 1")| (Q == "d2" & Q36 == "Opción 2") | \
            (Q == "d3" & Q36 == "Ninguno") | (Q == "e1" & Q38 == "Opción 1")| (Q == "e2" & Q38 == "Opción 2") | (Q == "e3" & Q38 == "Ninguno") | \
                (Q == "f1" & Q40 == "Opción 1")| (Q == "f2" & Q40 == "Opción 2") | (Q == "f3" & Q40 == "Ninguno") | (Q == "g1" & Q42 == "Opción 1")| \
                    (Q == "g2" & Q42 == "Opción 2") | (Q == "g3" & Q42 == "Ninguno") | (Q == "h1" & Q44 == "Opción 1")| (Q == "h2" & Q44 == "Opción 2") | \
                        (Q == "h3" & Q44 == "Ninguno") | (Q == "i1" & Q46 == "Opción 1")| (Q == "i2" & Q46 == "Opción 2") | (Q == "i3" & Q46 == "Ninguno") | \
                            (Q == "j1" & Q48 == "Opción 1")| (Q == "j2" & Q48 == "Opción 2") | (Q == "j3" & Q48 == "Ninguno") | (Q == "k1" & Q50 == "Opción 1")| \
                                (Q == "k2" & Q50 == "Opción 2") | (Q == "k3" & Q50 == "Ninguno") | (Q == "l1" & Q52 == "Opción 1")| (Q == "l2" & Q52 == "Opción 2") | \
                                    (Q == "l3" & Q52 == "Ninguno")').index, 'Choice'] = 1

data2.iloc[:40,-8:]

In [ ]:
# select the columns you want to move to the front
columns_to_move = ['Block', 'Set', 'Alt']
df_to_move = data2[columns_to_move]

# concatenate the selected columns with the rest of the dataframe
data2 = pd.concat([df_to_move, data2.drop(columns_to_move, axis=1)], axis=1)


In [ ]:
# Save the dataset as a .csv file named "main_dataset_py.csv" in the \data_output directory

#data2.to_csv("D:\OneDrive - Texas Tech University\PUBLICATIONS\In Progress\complex_supply_chain\data_output\main_dataset_py.csv", index=False)

### The Second part of the script is about the setup of the experimental design in the right format

In [ ]:
# Import data of the experimental design into a pandas dataframe

design = pd.read_stata("D:\OneDrive - Texas Tech University\PUBLICATIONS\In Progress\complex_supply_chain\data_output\exp_design.dta")

#design.head()

In [ ]:
# Create dummy variables for each stage of the supply chain

# create dummies for "born" column
dummy_born = pd.get_dummies(design['born'], prefix='Born')
dummy_born.rename(columns={'Born_mx': 'Born_mx', 
                           'Born_us': 'Born_us', 
                           'Born_tx': 'Born_tx', 
                           'Born_can': 'Born_can', 
                           'Born_nic': 'Born_nic'}, inplace=True)

# create dummies for "raised" column
raised_dummies = pd.get_dummies(design["raised"], prefix="Raised")
raised_dummies.rename(columns={"Raised_mx": "Raised_mx",
                               "Raised_us": "Raised_us",
                               "Raised_tx": "Raised_tx",
                               "Raised_can": "Raised_can",
                               "Raised_nic": "Raised_nic"}, inplace=True)

# create dummies for "slaughtered" column
slaughtered_dummies = pd.get_dummies(design["slaughtered"], prefix="Slaughtered")
slaughtered_dummies.rename(columns={"Slaughtered_mx": "Slaughtered_mx",
                                     "Slaughtered_us": "Slaughtered_us",
                                     "Slaughtered_tx": "Slaughtered_tx",
                                     "Slaughtered_can": "Slaughtered_can",
                                     "Slaughtered_nic": "Slaughtered_nic"}, inplace=True)


# create a dummy variable for "fsafety" where 0=Standard and 1=Enhanced
foodsafety_dummies = pd.get_dummies(design["fsafety"], prefix="Fsafety")
foodsafety_dummies .rename(columns={"Fsafety": "Fsafety"}, inplace=True)

# create a dummy variable for "prod" where 0=Approved and 1=Natural production practices
prod_dummies = pd.get_dummies(design["prod"], prefix="Prod")
prod_dummies.rename(columns={"Prod": "Prod"}, inplace=True)

# concatenate the three dummy dataframes with the original dataframe
design = pd.concat([design, dummy_born, raised_dummies, slaughtered_dummies, foodsafety_dummies, prod_dummies], axis=1)

In [ ]:

# rename the new variables if needed
design = design.rename(columns={'block':'Block', 'set':'Set', 'alt':'Alt', 'price':'Price', \
    'Born_BMexico':'Born_mx', 'Born_BUS':'Born_us', 'Born_BUS-TX':'Born_tx', 'Born_BCanada':'Born_can', 'Born_BNicaragua':'Born_nic', \
        'Raised_RMexico':'Raised_mx', 'Raised_RUS':'Raised_us', 'Raised_RUS-TX':'Raised_tx', 'Raised_RCanada':'Raised_can', 'Raised_RNicaragua':'Raised_nic', \
            'Slaughtered_1.0':'Slaughtered_mx', 'Slaughtered_2.0':'Slaughtered_us', 'Slaughtered_3.0':'Slaughtered_tx', 'Slaughtered_4.0':'Slaughtered_can', 'Slaughtered_5.0':'Slaughtered_nic', \
                })

# insert a new column named "ASC" filled with zeros
design['ASC'] = 0

# drop the unnecesary columns in the dataframe
design = design.drop('Prod_Approved', axis=1).drop('Fsafety_Standard', axis=1).drop('born', axis=1).drop('raised', axis=1).drop('slaughtered', axis=1).drop('fsafety', axis=1).drop('prod', axis=1)

# remove the "$" sign from the values in the 'price' column
design['Price'] = design['Price'].str.replace('$','', regex=False)

# convert the 'price' column to numeric data type
design['Price'] = pd.to_numeric(design['Price'])

#design.iloc[:10,-20:]

# convert the dataframe values to integers
design = design.astype('int64')
#print(design.dtypes)

design

In [ ]:
# create a new dataframe
columns = design.columns
df_new = pd.DataFrame(np.zeros((36, len(columns))), columns=columns).astype('int64')

df_new['Block'] = (df_new.index) // 12 + 1
df_new['Set'] = 1 + ((df_new.index) % 12)
df_new['Alt'] = 3
df_new['ASC'] = 1

print(df_new)

In [ ]:
design2 = pd.concat([design, df_new])
design2
#design2.iloc[:,-20:]

In [ ]:
# sort data by Block and Alt
design2 = design2.sort_values(by=['Block', 'Set', 'Alt', 'ASC'])


In [ ]:
# select the columns you want to move to the front
columns_to_move2 = ['Block', 'Set', 'Alt']
df_to_move2 = design2[columns_to_move2]

# concatenate the selected columns with the rest of the dataframe
design2 = pd.concat([df_to_move2, design2.drop(columns_to_move2, axis=1)], axis=1)

design2.head().T

In [ ]:
# Save the dataset as a .csv file named "dataset_design_py.csv" in the \data_output directory

#design2.to_csv("D:\OneDrive - Texas Tech University\PUBLICATIONS\In Progress\complex_supply_chain\data_output\dataset_design_py.csv", index=False)

### The Third part of the script is about the setup of the dataset 

In [ ]:
# Load main dataset
main_dataset = pd.read_csv("D:\OneDrive - Texas Tech University\PUBLICATIONS\In Progress\complex_supply_chain\data_output\main_dataset_py.csv", low_memory=False)

# Load experimental design dataset
main_design = pd.read_csv("D:\OneDrive - Texas Tech University\PUBLICATIONS\In Progress\complex_supply_chain\data_output\dataset_design_py.csv", low_memory=False)

# Check if the three columns 'Block', 'Set', and 'Alt' exist in both dataframes
if all(col in main_dataset.columns for col in ['Block', 'Set', 'Alt']) and all(col in main_design.columns for col in ['Block', 'Set', 'Alt']):
    # Combine the three columns into one in both dataframes
    main_dataset['Block Set Alt'] = main_dataset['Block'].astype(str) + ' ' + main_dataset['Set'].astype(str) + ' ' + main_dataset['Alt'].astype(str)
    main_design['Block Set Alt'] = main_design['Block'].astype(str) + ' ' + main_design['Set'].astype(str) + ' ' + main_design['Alt'].astype(str)
    
    # Merge the two datasets using a one-to-one (1:1) merge on "Block Set Alt"
    merged_dataset = main_dataset.merge(main_design, on='Block Set Alt', how='left')
else:
    print("Error: Columns 'Block', 'Set', or 'Alt' not found in one of the dataframes.")


In [ ]:
merged_dataset.iloc[:, :20]

In [ ]:
# sort data by Block, ID, Set, and Alt
merged_dataset = merged_dataset.sort_values(by=['Block_x', 'ID', 'Set_x','Alt_x'])

# Get the size of the dataframe
rows, columns = merged_dataset.shape
print("The dataframe has {} rows and {} columns".format(rows, columns))

#merged_dataset.iloc[:, -25:]

In [ ]:
# create a new variable named "CHOICESITUATION" that calculate the floor division of the row index divided by 3,
merged_dataset['CHOICESITUATION'] = np.floor((merged_dataset.index)/3).astype(int) + 1

#merged_dataset.iloc[:, -12:]

In [ ]:
# rename the new variables if needed
merged_dataset = merged_dataset.rename(columns={'Block_x':'Block', 'Set_x':'Set', 'Alt_x':'Alt'})

# drop the unnecesary columns in the dataframe
merged_dataset = merged_dataset.drop('Block_y', axis=1).drop('Set_y', axis=1).drop('Alt_y', axis=1).drop('Block Set Alt', axis=1)

# select the columns you want to move to the front
columns_to_move3 = ['CHOICESITUATION', 'Block', 'Set', 'Alt', 'Choice', 'ASC', 'ID', 'Born_mx', 'Born_us', 'Born_tx', 'Born_can', 'Born_nic', 'Raised_mx', \
    'Raised_us', 'Raised_tx', 'Raised_can', 'Raised_nic', 'Slaughtered_mx', 'Slaughtered_us', 'Slaughtered_tx', 'Slaughtered_can', 'Slaughtered_nic', \
        'Price', 'Fsafety_Enhanced','Prod_Natural']

df_to_move3 = merged_dataset[columns_to_move3]

# concatenate the selected columns with the rest of the dataframe
merged_dataset = pd.concat([df_to_move3, merged_dataset.drop(columns_to_move3, axis=1)], axis=1)

In [ ]:
# Get the size of the dataframe
rows, columns = merged_dataset.shape
print("The dataframe has {} rows and {} columns".format(rows, columns))

In [ ]:
merged_dataset.iloc[:, :-10]

In [ ]:
# Save the dataset as a .csv file named "dataset_py.csv" in the \data_output directory

#merged_dataset.to_csv("D:\OneDrive - Texas Tech University\PUBLICATIONS\In Progress\complex_supply_chain\data_output\dataset_py.csv", index=False)

### The Forth and last part of the script and it is about the Willingness to pay analysis

In [ ]:
# The Iterable abstract class was removed from collections in Python 3.10. We can use Iterable from collections.abc instead, or use Python 3.9

!pip install pylogit
!pip install biogeme
#from collections.abc import Iterable

import pandas as pd                    # For file input/output
import numpy as np                     # For vectorized math operations
#import pylogit as pl                   # For MNL model estimation and conversion from wide to long format


In [ ]:
# Load the dataset
dataset = pd.read_csv("D:\OneDrive - Texas Tech University\PUBLICATIONS\In Progress\complex_supply_chain\data_output\dataset_py.csv", low_memory=False)

In [ ]:
# Look at the first 3 rows of the data transposed
#dataset.head(3).T

In [ ]:
''' Steps

1. Install biogeme by running !pip install biogeme in your Python environment.

2. Prepare your dataset with the following columns:
    An alternative ID column
    A choice column (1 if the alternative was chosen, 0 otherwise)
    Columns for each attribute of the alternatives
    A column for the WTP associated with each attribute

3. Define the model specification using the biogeme language, which is similar to the Stata syntax. Here is an example:

'''

ASC1 = Beta('ASC1', 0, None, None, 0)
ASC2 = Beta('ASC2', 0, None, None, 1)
B_PRICE = Beta('B_PRICE', 0, None, None, 0)
B_QUALITY = Beta('B_QUALITY', 0, None, None, 0)
B_WTP_PRICE = Beta('B_WTP_PRICE', 0, None, None, 0)
B_WTP_QUALITY = Beta('B_WTP_QUALITY', 0, None, None, 0)

MU = Beta('MU', 1, 0, None, 0)

V1 = ASC1 + B_PRICE * PRICE + B_QUALITY * QUALITY + B_WTP_PRICE * WTP_PRICE1 + B_WTP_QUALITY * WTP_QUALITY1
V2 = ASC2 + B_PRICE * PRICE + B_QUALITY * QUALITY + B_WTP_PRICE * WTP_PRICE2 + B_WTP_QUALITY * WTP_QUALITY2

V = {1: V1, 2: V2}
av = {1: 1, 2: 1}

prob = bioLogit(V, av, CHOICE)
logprob = log(MonteCarlo(prob))

loglike = sum(logprob)

In [ ]:
''' cont.

In this example, PRICE and QUALITY are the attribute columns, and WTP_PRICE1, WTP_QUALITY1, WTP_PRICE2, and WTP_QUALITY2 are the WTP columns. 
ASC1 and ASC2 are alternative-specific constants, B_PRICE, B_QUALITY, B_WTP_PRICE, and B_WTP_QUALITY are the model parameters, and MU is the scale parameter. 
CHOICE is the column with the binary choice variable.

4. Run the model estimation using the following code:

'''
model = bioModel(loglike)
results = model.estimate()

'''

5. View the results using results.getEstimatedParameters(). 
This will give you the estimated values of the model parameters and their standard errors, t-statistics, and p-values. 
You can also use results.getGeneralStatistics() to view other model statistics, such as the log-likelihood and the number of observations.

'''

